#### 다중인덱스, 피봇, 그룹

In [3]:
import numpy  as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

from datetime import date, datetime, timedelta
from dateutil.parser import parse

import json
import urllib 

# titanic dataset
import seaborn as sns 

# 비정형 데이터 처리를 위한 라이브러리
from bs4 import BeautifulSoup
import requests 
import re 



print('numpy  version - ' , np.__version__)
print('pandas version - ' , pd.__version__)

numpy  version -  1.20.3
pandas version -  1.3.4


In [4]:
def seriesInfo(s) :
    print('series : index + values \n' , s) 
    print('index  - ' , s.index  , type(s.index))
    print('values - ' , s.values , type(s.values))

def frmInfo(df) :
    display('data - ' , df )
    print()
    print('shape   - ' , df.shape , df.shape[0] , df.shape[1])
    print('size    - ' , df.size )
    print('ndim    - ' , df.ndim)
    print('index   - ' , df.index    , type(df.index))
    print('columns - ' , df.columns  , type( df.columns ))
    print('values  - \n' , df.values , type(df.values) )

- 다중인덱스 (열)
- 여러개의 계층을 가지는 인덱스를 의미한다
- 사용의 편리성을 위해서 열 인덱스 이름 부여하여 사용할 수 있다

In [5]:
multi_col_idx_frm = pd.DataFrame( np.round( np.random.randn(5, 4) , 2 )    , 
                                  columns = [
                                      ['Grp01' , 'Grp01' , 'Grp02' , 'Grp02'] ,
                                      ['col01' , 'col02' , 'col03' , 'col04']
                                  ])

multi_col_idx_frm

Grp01       Grp02      
  col01 col02 col03 col04
0  0.87  0.62  0.94 -1.30
1 -0.66 -0.98  0.29 -0.42
2 -0.47 -0.98  2.14 -0.30
3  1.64  2.14  0.33  0.60
4  0.02 -0.58  1.27  0.90

In [6]:
multi_col_idx_frm.columns

MultiIndex([('Grp01', 'col01'),
            ('Grp01', 'col02'),
            ('Grp02', 'col03'),
            ('Grp02', 'col04')],
           )

In [7]:
multi_col_idx_frm.columns.names = ['GrpIdx' , 'ColIdx' ]
multi_col_idx_frm

GrpIdx Grp01       Grp02      
ColIdx col01 col02 col03 col04
0       0.87  0.62  0.94 -1.30
1      -0.66 -0.98  0.29 -0.42
2      -0.47 -0.98  2.14 -0.30
3       1.64  2.14  0.33  0.60
4       0.02 -0.58  1.27  0.90

In [13]:
multi_col_idx_frm.columns.names

FrozenList(['GrpIdx', 'ColIdx'])

- (행)다중 인덱스

In [15]:
multi_idx_frm = pd.DataFrame( np.round( np.random.randn(6, 4) , 2 )    , 
                                  columns = [
                                      ['Grp01' , 'Grp01' , 'Grp02' , 'Grp02'] ,
                                      ['col01' , 'col02' , 'col03' , 'col04']
                                  ] , 
                                  index = [
                                      ['M' , 'M' , 'M' , 'F' , 'F' , 'F'] , 
                                      [ 'id_' + str(idx) for idx in range(6) ] 
                                  ])

multi_idx_frm.columns.names = ['GrpIdx'  , 'ColIdx' ]
multi_idx_frm.index.names   = ['GenderIdx' , 'RowIdx' ]
multi_idx_frm

GrpIdx           Grp01       Grp02      
ColIdx           col01 col02 col03 col04
GenderIdx RowIdx                        
M         id_0   -0.85 -0.46  0.48  0.67
          id_1    0.28  0.01 -1.26  1.42
          id_2   -0.06  0.07  0.66  0.27
F         id_3    1.59 -2.52 -0.19  0.34
          id_4    1.82  0.02 -1.56  2.64
          id_5    0.78 -0.85 -0.14  0.45

- stack()   : 열 인덱스 -> 행 인덱스
- unstack() : 행 인덱스 -> 열 인덱스

In [18]:
# multi_idx_frm.stack()
multi_idx_frm.stack('GrpIdx')

ColIdx                   col01  col02  col03  col04
GenderIdx RowIdx GrpIdx                            
M         id_0   Grp01   -0.85  -0.46    NaN    NaN
                 Grp02     NaN    NaN   0.48   0.67
          id_1   Grp01    0.28   0.01    NaN    NaN
                 Grp02     NaN    NaN  -1.26   1.42
          id_2   Grp01   -0.06   0.07    NaN    NaN
                 Grp02     NaN    NaN   0.66   0.27
F         id_3   Grp01    1.59  -2.52    NaN    NaN
                 Grp02     NaN    NaN  -0.19   0.34
          id_4   Grp01    1.82   0.02    NaN    NaN
                 Grp02     NaN    NaN  -1.56   2.64
          id_5   Grp01    0.78  -0.85    NaN    NaN
                 Grp02     NaN    NaN  -0.14   0.45

In [20]:
# multi_idx_frm.unstack()
multi_idx_frm.unstack('GenderIdx')

GrpIdx    Grp01                   Grp02                  
ColIdx    col01       col02       col03       col04      
GenderIdx     F     M     F     M     F     M     F     M
RowIdx                                                   
id_0        NaN -0.85   NaN -0.46   NaN  0.48   NaN  0.67
id_1        NaN  0.28   NaN  0.01   NaN -1.26   NaN  1.42
id_2        NaN -0.06   NaN  0.07   NaN  0.66   NaN  0.27
id_3       1.59   NaN -2.52   NaN -0.19   NaN  0.34   NaN
id_4       1.82   NaN  0.02   NaN -1.56   NaN  2.64   NaN
id_5       0.78   NaN -0.85   NaN -0.14   NaN  0.45   NaN

- 다중 인덱스의 인덱싱은 어떻게 ? 
- 튜플형식으로 접근하면 된다

In [29]:
# multi_idx_frm
# multi_idx_frm.columns

multi_idx_frm[ [ ('Grp01', 'col01') , ('Grp01', 'col02') ] ]

GrpIdx           Grp01      
ColIdx           col01 col02
GenderIdx RowIdx            
M         id_0   -0.85 -0.46
          id_1    0.28  0.01
          id_2   -0.06  0.07
F         id_3    1.59 -2.52
          id_4    1.82  0.02
          id_5    0.78 -0.85

In [30]:
multi_idx_frm

GrpIdx           Grp01       Grp02      
ColIdx           col01 col02 col03 col04
GenderIdx RowIdx                        
M         id_0   -0.85 -0.46  0.48  0.67
          id_1    0.28  0.01 -1.26  1.42
          id_2   -0.06  0.07  0.66  0.27
F         id_3    1.59 -2.52 -0.19  0.34
          id_4    1.82  0.02 -1.56  2.64
          id_5    0.78 -0.85 -0.14  0.45

In [36]:
multi_idx_frm[('Grp02','col03')].values[0]

0.48

In [45]:
multi_idx_frm.loc[('M', 'id_1'), 'Grp02']
multi_idx_frm['Grp02'].values[1]
# multi_idx_frm[ ('Grp02','col03') ]  [('M','id_2')]
multi_idx_frm[[('Grp02','col03'),('Grp02','col04')]].iloc[1].values



array([-1.26,  1.42])

In [49]:
print('M 정보만 출력 - ')

multi_idx_frm.loc['M']
multi_idx_frm.iloc[ : 3 , 2 : 4  ]

M 정보만 출력 - 


GrpIdx           Grp02      
ColIdx           col03 col04
GenderIdx RowIdx            
M         id_0    0.48  0.67
          id_1   -1.26  1.42
          id_2    0.66  0.27

- 다중 인덱스 정렬
- 축 지정 axis
- level 기준을 정의하여야 한다

In [59]:
multi_idx_frm

GrpIdx           Grp01       Grp02      
ColIdx           col01 col02 col03 col04
GenderIdx RowIdx                        
M         id_0   -0.85 -0.46  0.48  0.67
          id_1    0.28  0.01 -1.26  1.42
          id_2   -0.06  0.07  0.66  0.27
F         id_3    1.59 -2.52 -0.19  0.34
          id_4    1.82  0.02 -1.56  2.64
          id_5    0.78 -0.85 -0.14  0.45

In [67]:
multi_idx_frm.sort_index(level = 1 , axis = 1)

GrpIdx           Grp01       Grp02      
ColIdx           col01 col02 col03 col04
GenderIdx RowIdx                        
M         id_0   -0.85 -0.46  0.48  0.67
          id_1    0.28  0.01 -1.26  1.42
          id_2   -0.06  0.07  0.66  0.27
F         id_3    1.59 -2.52 -0.19  0.34
          id_4    1.82  0.02 -1.56  2.64
          id_5    0.78 -0.85 -0.14  0.45

- 그룹
- groupby(열 | 행) 
- 그룹연산( size , count , mean , median , min , max , std , var , quantile , first , last , aggregate etc......) 


In [68]:
grp_frm = pd.DataFrame({
    '학과' : ['인공지능' , '데이터분석' , '인공지능' , '데이터분석' , '데이터분석'] , 
    '학년' : [1, 2, 3, 4, 2] , 
    '이름' : ['섭섭해' , '임섭순' , '최강희' , '홍길동' , '신사임당'] , 
    '학점' : [2.7, 3.5, 4.5, 3.8, 4.3]
})

grp_frm

,학과,학년,이름,학점
0,인공지능,1,섭섭해,2.7
1,데이터분석,2,임섭순,3.5
2,인공지능,3,최강희,4.5
3,데이터분석,4,홍길동,3.8
4,데이터분석,2,신사임당,4.3


In [73]:
print('그룹을 만드는 방법은 두가지로 데이터프레임그룹 과 시리즈그룹이 생성될 수 있다. - ')
dept_frm = grp_frm.groupby( grp_frm['학과'])
print('프레임 그룹 - ' , dept_frm )

dept_series = grp_frm['학과'].groupby( grp_frm['학과'])
print('프레임 그룹 - ' , dept_series )



그룹을 만드는 방법은 두가지로 데이터프레임그룹 과 시리즈그룹이 생성될 수 있다. - 
프레임 그룹 -  <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001B2CDD18F40>
프레임 그룹 -  <pandas.core.groupby.generic.SeriesGroupBy object at 0x000001B2CDF566D0>


In [80]:
print('그룹 정보 확인 - ')
print('groups - ' ,  dept_series.groups )
print()
print('그룹 정보를 가져오고 싶다면 - ') 
print(dept_series.get_group('데이터분석') )
print()
print(dept_series.get_group('인공지능') )

그룹 정보 확인 - 
groups -  {'데이터분석': [1, 3, 4], '인공지능': [0, 2]}

그룹 정보를 가져오고 싶다면 - 
1    데이터분석
3    데이터분석
4    데이터분석
Name: 학과, dtype: object

0    인공지능
2    인공지능
Name: 학과, dtype: object


In [86]:
print('데이터프레임 그룹 - ')

print('그룹 정보 확인 - ')
print('groups - ' ,  dept_frm.groups )
print()
print('그룹 정보를 가져오고 싶다면 - ') 
display(dept_frm.get_group('데이터분석') )
print()
display(dept_frm.get_group('인공지능') )

데이터프레임 그룹 - 
그룹 정보 확인 - 
groups -  {'데이터분석': [1, 3, 4], '인공지능': [0, 2]}

그룹 정보를 가져오고 싶다면 - 


,학과,학년,이름,학점
1,데이터분석,2,임섭순,3.5
3,데이터분석,4,홍길동,3.8
4,데이터분석,2,신사임당,4.3


,학과,학년,이름,학점
0,인공지능,1,섭섭해,2.7
2,인공지능,3,최강희,4.5


In [89]:
dept_frm.describe()

학년                                                 학점            \
      count      mean       std  min  25%  50%  75%  max count      mean   
학과                                                                         
데이터분석   3.0  2.666667  1.154701  2.0  2.0  2.0  3.0  4.0   3.0  3.866667   
인공지능    2.0  2.000000  1.414214  1.0  1.5  2.0  2.5  3.0   2.0  3.600000   

                                            
            std  min   25%  50%   75%  max  
학과                                          
데이터분석  0.404145  3.5  3.65  3.8  4.05  4.3  
인공지능   1.272792  2.7  3.15  3.6  4.05  4.5

In [112]:
print('학과별 학점 평균만 출력한다면 - ')
# dept_frm.mean()
print('case01 - ')
display( grp_frm['학점'].groupby(grp_frm['학과']).mean() )
print()
print('case02 - ') 
print( grp_frm.groupby(grp_frm['학과'])['학점'].mean() )
print()
print('case03 - ') 
print( grp_frm.groupby(grp_frm['학과']).mean()['학점'] )



학과별 학점 평균만 출력한다면 - 
case01 - 


학과
데이터분석    3.866667
인공지능     3.600000
Name: 학점, dtype: float64


case02 - 
학과
데이터분석    3.866667
인공지능     3.600000
Name: 학점, dtype: float64

case03 - 
학과
데이터분석    3.866667
인공지능     3.600000
Name: 학점, dtype: float64


In [91]:
dept_frm.sum()

,학년,학점
학과,,
데이터분석,8,11.6
인공지능,4,7.2


In [92]:
dept_frm.agg( [np.mean , np.sum] )

학년            학점      
           mean sum      mean   sum
학과                                 
데이터분석  2.666667   8  3.866667  11.6
인공지능   2.000000   4  3.600000   7.2

- 다중그룹

In [96]:
grp_frm

,학과,학년,이름,학점
0,인공지능,1,섭섭해,2.7
1,데이터분석,2,임섭순,3.5
2,인공지능,3,최강희,4.5
3,데이터분석,4,홍길동,3.8
4,데이터분석,2,신사임당,4.3


In [97]:
print('다중 그룹 - ')

multi_grp_frm = grp_frm.groupby(['학과' , '학년'])

print('그룹 정보 확인 - ')
print('groups - ' ,  multi_grp_frm.groups )

그룹 정보 확인 - 
groups -  {('데이터분석', 2): [1, 4], ('데이터분석', 4): [3], ('인공지능', 1): [0], ('인공지능', 3): [2]}


In [98]:
tmp_frm = pd.DataFrame({
    'idx01' : ['A' , 'A' , 'B' , 'B' , 'A'] , 
    'idx02' : ['one' , 'two' , 'one' , 'two' , 'one'] , 
    'idx01_data01' : [1, 2, 3, 4, 5] , 
    'idx02_data02' : [10, 20, 30, 40, 50] ,    
})

tmp_frm


,idx01,idx02,idx01_data01,idx02_data02
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [100]:
print('idx01 그룹을 만들고 싶다면 - ') 
idx01_grp = tmp_frm.groupby(tmp_frm['idx01'])
idx01_grp.groups

idx01 그룹을 만들고 싶다면 - 


{'A': [0, 1, 4], 'B': [2, 3]}

In [102]:
print('그룹에 대한 수치값들만 그룹 연산이 된다 - ') 
idx01_grp.sum()

그룹에 대한 수치값들만 그룹 연산이 된다 - 


,idx01_data01,idx02_data02
idx01,,
A,8,80
B,7,70


In [104]:
tmp_frm['idx01_data01'].groupby(tmp_frm['idx01']).mean()

idx01
A    2.666667
B    3.500000
Name: idx01_data01, dtype: float64

In [116]:
print('다중 키 또는 복합 키 - ')
tmp_frm['idx01_data01'].groupby([tmp_frm['idx01'] , tmp_frm['idx02']]).sum().unstack()

다중 키 또는 복합 키 - 


idx02,one,two
idx01,,
A,6,2
B,3,4


In [5]:
titanic_dataset = sns.load_dataset('titanic')
iris_dataset = sns.load_dataset('iris')

In [118]:
titanic_dataset

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [119]:
iris_dataset

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [121]:
print('age, sex, class, fare, survived - 순서로 서브셋 구축')

# subset = titanic_dataset[['age','sex','class','fare','survived']]
subset = titanic_dataset.loc[ : , ['age','sex','class','fare','survived']]
subset

age, sex, class, fare, survived - 순서로 서브셋 구축


,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0
...,...,...,...,...,...
886,27.0,male,Second,13.0000,0
887,19.0,female,First,30.0000,1
888,NaN,female,Third,23.4500,0
889,26.0,male,First,30.0000,1


In [122]:
subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       714 non-null    float64 
 1   sex       891 non-null    object  
 2   class     891 non-null    category
 3   fare      891 non-null    float64 
 4   survived  891 non-null    int64   
dtypes: category(1), float64(2), int64(1), object(1)
memory usage: 29.0+ KB


In [133]:
print('요구사항 - 선실등급에 따른 그룹을 생성한다면 ')
class_grp = subset.groupby(subset['class'])

for grp_id  , data in class_grp : 
    print('grp_id - ' , grp_id)
    print()
    print(data)

요구사항 - 선실등급에 따른 그룹을 생성한다면 
grp_id -  First

      age     sex  class     fare  survived
1    38.0  female  First  71.2833         1
3    35.0  female  First  53.1000         1
6    54.0    male  First  51.8625         0
11   58.0  female  First  26.5500         1
23   28.0    male  First  35.5000         1
..    ...     ...    ...      ...       ...
871  47.0  female  First  52.5542         1
872  33.0    male  First   5.0000         0
879  56.0  female  First  83.1583         1
887  19.0  female  First  30.0000         1
889  26.0    male  First  30.0000         1

[216 rows x 5 columns]
grp_id -  Second

      age     sex   class     fare  survived
9    14.0  female  Second  30.0708         1
15   55.0  female  Second  16.0000         1
17    NaN    male  Second  13.0000         1
20   35.0    male  Second  26.0000         0
21   34.0    male  Second  13.0000         1
..    ...     ...     ...      ...       ...
866  27.0  female  Second  13.8583         1
874  28.0  female  Second 

In [137]:
print('요구사항 - 선실등급에 따른 평균요금만 출력 - ' ) 

class_grp.mean()[['fare']]


요구사항 - 선실등급에 따른 평균요금만 출력 - 


,fare
class,
First,84.154687
Second,20.662183
Third,13.675550


In [248]:
print('요구사항 - 선실등급과 성별에 따른 그룹을 생성한다면 - ' ) 
class_sex_grp = subset.groupby(['class' , 'sex']) 

# for grp_id  , data in class_sex_grp : 
#     print('grp_id - ' , grp_id)
#     print()
#     print(data)
    
class_sex_grp.get_group(('First', 'female'))

요구사항 - 선실등급과 성별에 따른 그룹을 생성한다면 - 


,age,sex,class,fare,survived
1,38.0,female,First,71.2833,1
3,35.0,female,First,53.1000,1
11,58.0,female,First,26.5500,1
31,NaN,female,First,146.5208,1
52,49.0,female,First,76.7292,1
...,...,...,...,...,...
856,45.0,female,First,164.8667,1
862,48.0,female,First,25.9292,1
871,47.0,female,First,52.5542,1
879,56.0,female,First,83.1583,1


In [145]:
iris_dataset['species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [147]:
iris_species_grp = iris_dataset.groupby('species')
for grp_id  , data in iris_species_grp : 
    print('grp_id - ' , grp_id)
    print()
    print(data)
    


grp_id -  setosa

    sepal_length  sepal_width  petal_length  petal_width species
0            5.1          3.5           1.4          0.2  setosa
1            4.9          3.0           1.4          0.2  setosa
2            4.7          3.2           1.3          0.2  setosa
3            4.6          3.1           1.5          0.2  setosa
4            5.0          3.6           1.4          0.2  setosa
5            5.4          3.9           1.7          0.4  setosa
6            4.6          3.4           1.4          0.3  setosa
7            5.0          3.4           1.5          0.2  setosa
8            4.4          2.9           1.4          0.2  setosa
9            4.9          3.1           1.5          0.1  setosa
10           5.4          3.7           1.5          0.2  setosa
11           4.8          3.4           1.6          0.2  setosa
12           4.8          3.0           1.4          0.1  setosa
13           4.3          3.0           1.1          0.1  setosa
14     

In [148]:
iris_species_grp.get_group('setosa')

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [150]:
print('요구사항 - 품종별 가장큰 값과 가장 작은 값의 차이를 출력한다면 - ') 

def get_subtraction(x) :
    return x.max() - x.min()

print('요구사항 - 품종별 가장큰 값과 가장 작은 값의 비율을 출력한다면 - ') 

def get_ratio(x) :
    return x.max() / x.min()

요구사항 - 품종별 가장큰 값과 가장 작은 값의 차이를 출력한다면 - 
요구사항 - 품종별 가장큰 값과 가장 작은 값의 비율을 출력한다면 - 


In [155]:
iris_species_grp.agg([get_subtraction, get_ratio])

sepal_length               sepal_width            \
           get_subtraction get_ratio get_subtraction get_ratio   
species                                                          
setosa                 1.5  1.348837             2.1  1.913043   
versicolor             2.1  1.428571             1.4  1.700000   
virginica              3.0  1.612245             1.6  1.727273   

              petal_length               petal_width            
           get_subtraction get_ratio get_subtraction get_ratio  
species                                                         
setosa                 0.9  1.900000             0.5  6.000000  
versicolor             2.1  1.700000             0.8  1.800000  
virginica              2.4  1.533333             1.1  1.785714

In [173]:
print('요구사항 - 품종별 가장 큰 petal_length 3개 출력 - ') 

display( iris_dataset.sort_values(by='petal_length' , ascending = False).groupby('species').head(3) )

def petal(frm) :
    return frm.sort_values(by='petal_length' , ascending = False)[ : 3] 

요구사항 - 품종별 가장 큰 petal_length 3개 출력 - 


,sepal_length,sepal_width,petal_length,petal_width,species
118,7.7,2.6,6.9,2.3,virginica
122,7.7,2.8,6.7,2.0,virginica
117,7.7,3.8,6.7,2.2,virginica
83,6.0,2.7,5.1,1.6,versicolor
77,6.7,3.0,5.0,1.7,versicolor
52,6.9,3.1,4.9,1.5,versicolor
24,4.8,3.4,1.9,0.2,setosa
44,5.1,3.8,1.9,0.4,setosa
5,5.4,3.9,1.7,0.4,setosa


In [174]:
iris_species_grp.apply(petal)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

In [180]:
iris_species_grp.describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

#### pivot
- 데이터프레임에서 두개의 열을 이용하여 행/열 reshape 새로운 프레임을 만들 때
- pivot(index , columns , data.values)
- pivot_table(data , values , index , columns , aggfuc = , fill_value =  , margins = , margins_names = ) 

In [196]:
class_sex_subset_frm = titanic_dataset.loc[ : , ['pclass' , 'sex']]
class_sex_subset_frm
print()
class_sex_subset_frm = pd.DataFrame(titanic_dataset , 
                                    columns = ['pclass' , 'sex'])
class_sex_subset_frm

,pclass,sex
0,3,male
1,1,female
2,3,female
3,1,female
4,3,male
...,...,...
886,2,male
887,1,female
888,3,female
889,1,male


In [222]:
print('요구사항 - 성별과 객실등급에 따른 승객 수 집계한다면? - ')
print('groupby - ')

# class_sex_series = titanic_dataset.groupby(['pclass' , 'sex']).size()
# class_sex_series 

class_sex_cnt_frm = class_sex_subset_frm.groupby(['sex' , 'pclass']).size().reset_index(name='cnt')
class_sex_cnt_frm

print('pivot - ') 
class_sex_cnt_frm.pivot('sex' , 'pclass' , 'cnt')

요구사항 - 성별과 객실등급에 따른 승객 수 집계한다면? - 
groupby - 
pivot - 


pclass,1,2,3
sex,,,
female,94,76,144
male,122,108,347


In [226]:
print('요구사항 - 성별과 생존여부에 따른 승객 수 집계한다면? - ')
print('groupby - ')
sex_survived_frm = titanic_dataset.groupby(['sex' , 'survived']).size().reset_index(name = 'cnt')
display( sex_survived_frm )

print()
print('pivot - ')
sex_survived_frm.pivot('sex' , 'survived' , 'cnt')

요구사항 - 성별과 생존여부에 따른 승객 수 집계한다면? - 
groupby - 


,sex,survived,cnt
0,female,0,81
1,female,1,233
2,male,0,468
3,male,1,109



pivot - 


survived,0,1
sex,,
female,81,233
male,468,109


In [227]:
titanic_dataset['cnt'] = 1
titanic_dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,cnt
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,1
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,1
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,1
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,1


In [231]:
print('pivot_table - ') 
print()

titanic_dataset.pivot_table(index   = 'sex' , 
                            columns = 'survived' , 
                            values  = 'cnt' , 
                            aggfunc = np.sum , 
                            margins = True , 
                            margins_name = '집계')

pivot_table - 



survived,0,1,집계
sex,,,
female,81,233,314
male,468,109,577
집계,549,342,891


In [6]:
tips_dataset = sns.load_dataset('tips')
tips_dataset

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [25]:
print('요구사항 - 식사대금 대비 팁의 비율이 가장 높은 데이터를 찾는다면 - ')

print('case01 - ') 
def ratio(x) : 
    return x['tip'] / x['total_bill']

tips_dataset.groupby(['total_bill' , 'tip']).apply(ratio).reset_index(name='tip_ratio').sort_values(by='tip_ratio' , ascending = False)[0:1]


print('case02 - ')

tips_dataset['tip']
x = lambda frm: np.round((frm['tip'] / frm['total_bill'] * 100),2)
tips_dataset['tip_ratio'] = tips_dataset.apply(x,axis=1)
tips_dataset.sort_values('tip_ratio', ascending=False)

print('case03 - ')
tips_dataset['ratio'] = tips_dataset.apply(lambda x : x['tip'] / x['total_bill'], axis=1)
tips_dataset.iloc[tips_dataset['ratio'].argmax()]

print('case04 - teacher ^*^ ') 
tips_dataset['ratio'] = tips_dataset['tip'] / tips_dataset['total_bill']
print('max idx - ' , tips_dataset['ratio'].idxmax() )
tips_dataset.loc[ tips_dataset['ratio'].idxmax() , :  ]


요구사항 - 식사대금 대비 팁의 비율이 가장 높은 데이터를 찾는다면 - 
case01 - 
case02 - 
case03 - 
case04 - teacher ^*^ 
max idx -  172


total_bill        7.25
tip               5.15
sex               Male
smoker             Yes
day                Sun
time            Dinner
size                 2
tip_ratio        71.03
ratio         0.710345
Name: 172, dtype: object

In [28]:
print('성별에 따른 집계 - ')
print('count() , size()')
tips_dataset.groupby('sex').count()

성별에 따른 집계 - 
count() , size()


,total_bill,tip,smoker,day,time,size,tip_ratio,ratio
sex,,,,,,,,
Male,157,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87,87


In [32]:
print('성별과 흡연유무에 따른 팁 집계 - ')
tips_dataset.groupby(['sex' , 'smoker']).size() 


성별과 흡연유무에 따른 팁 집계 - 


sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [39]:
tips_dataset.pivot_table(index   = 'sex' , 
                         columns = 'smoker' , 
                         values  = 'ratio' ,
                         aggfunc = 'count' , 
                         margins = True , 
                         margins_name = '집계')

smoker,Yes,No,집계
sex,,,
Male,60,97,157
Female,33,54,87
집계,93,151,244


In [54]:
print('성별과 흡연 여부에 따른 평균 팁 비율을 확인한다면? - ') 

display( tips_dataset.groupby('sex')[['tip_ratio']].mean() )

display( tips_dataset.groupby('smoker')[['tip_ratio']].mean() )

display( tips_dataset.pivot_table(index   = 'sex' , 
                                  columns = 'smoker' ,
                                  values  = 'tip_ratio' , 
                                  aggfunc = 'mean' , 
                                  margins = True ) )

성별과 흡연 여부에 따른 평균 팁 비율을 확인한다면? - 


,tip_ratio
sex,
Male,15.764713
Female,16.648276


,tip_ratio
smoker,
Yes,16.319140
No,15.932318


smoker,Yes,No,All
sex,,,
Male,15.276667,16.066598,15.764713
Female,18.214545,15.691111,16.648276
All,16.319140,15.932318,16.079754


- 동일한 갯수 , 동일한 길이의 범주를 만들어 그룹에 대한 통계량을 확인할 수 있다.
- qcut() -> 동일한 갯수
- cut() -> 동일한 길이 

- transform() , apply() , agg()

In [65]:
print('iris - 붓꽃 꽃잎의 길이가 해당 품종내에서 대/중/소 어느그룹에 해당되는지 알고싶다면? - ') 
display( iris_dataset ) 
print()

iris_grp = iris_dataset.groupby('species')['petal_length']
for key, grp in iris_grp :
    print('key - ' , key)
    print('len - ' , len(grp))
    print()
    print(grp)
    print('*' * 50)

iris - 붓꽃 꽃잎의 길이가 해당 품종내에서 대/중/소 어느그룹에 해당되는지 알고싶다면? - 


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica



key -  setosa
len -  50

0     1.4
1     1.4
2     1.3
3     1.5
4     1.4
5     1.7
6     1.4
7     1.5
8     1.4
9     1.5
10    1.5
11    1.6
12    1.4
13    1.1
14    1.2
15    1.5
16    1.3
17    1.4
18    1.7
19    1.5
20    1.7
21    1.5
22    1.0
23    1.7
24    1.9
25    1.6
26    1.6
27    1.5
28    1.4
29    1.6
30    1.6
31    1.5
32    1.5
33    1.4
34    1.5
35    1.2
36    1.3
37    1.4
38    1.3
39    1.5
40    1.3
41    1.3
42    1.3
43    1.6
44    1.9
45    1.4
46    1.6
47    1.4
48    1.5
49    1.4
Name: petal_length, dtype: float64
**************************************************
key -  versicolor
len -  50

50    4.7
51    4.5
52    4.9
53    4.0
54    4.6
55    4.5
56    4.7
57    3.3
58    4.6
59    3.9
60    3.5
61    4.2
62    4.0
63    4.7
64    3.6
65    4.4
66    4.5
67    4.1
68    4.5
69    3.9
70    4.8
71    4.0
72    4.9
73    4.7
74    4.3
75    4.4
76    4.8
77    5.0
78    4.5
79    3.5
80    3.8
81    3.7
82    3.9
83    5.1
84    4.5
85    4.5

In [67]:
iris_dataset.groupby('species')['petal_length'].size()

species
setosa        50
versicolor    50
virginica     50
Name: petal_length, dtype: int64

In [98]:
def qcut_petal_length(s) :
    return pd.qcut(s , 3 , labels = ['대' , '중' , '소'])

In [99]:
iris_dataset['category'] = iris_dataset.groupby('species')['petal_length'].apply(qcut_petal_length)
iris_dataset

,sepal_length,sepal_width,petal_length,petal_width,species,category
0,5.1,3.5,1.4,0.2,setosa,대
1,4.9,3.0,1.4,0.2,setosa,대
2,4.7,3.2,1.3,0.2,setosa,대
3,4.6,3.1,1.5,0.2,setosa,중
4,5.0,3.6,1.4,0.2,setosa,대
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,대
146,6.3,2.5,5.0,1.9,virginica,대
147,6.5,3.0,5.2,2.0,virginica,대
148,6.2,3.4,5.4,2.3,virginica,중


In [89]:
print('품종별 꽃잎의 길이, 꽃잎의 너비의 평균을 구한다면 ? - ')
iris_dataset.groupby('species').mean()[['petal_length', 'petal_width']]
iris_dataset.groupby('species')[['petal_length', 'petal_width']].mean()

print()
iris_dataset.groupby('species').agg(np.mean).loc[ : , ['petal_length', 'petal_width'] ]

품종별 꽃잎의 길이, 꽃잎의 너비의 평균을 구한다면 ? - 



,petal_length,petal_width
species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [91]:
print('품종별로 최대/최소값을 구한다면? - ') 
iris_dataset.groupby('species').agg([np.min , np.max])

품종별로 최대/최소값을 구한다면? - 


sepal_length      sepal_width      petal_length      petal_width  \
                   amin amax        amin amax         amin amax        amin   
species                                                                       
setosa              4.3  5.8         2.3  4.4          1.0  1.9         0.1   
versicolor          4.9  7.0         2.0  3.4          3.0  5.1         1.0   
virginica           4.9  7.9         2.2  3.8          4.5  6.9         1.4   

                category       
           amax     amin amax  
species                        
setosa      0.6        대    소  
versicolor  1.8        대    소  
virginica   2.5        대    소

In [101]:
titanic_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
 15  category     714 non-null    category
dtypes: bool(2), category(3), float64(2), int64(4), object(5)
memory usage: 81.7+ KB


In [100]:
print('cut , qcut 이용해서 나이에대한 범주를 4개로 만들어서 새로운 피처 추가한다면? - ') 
titanic_dataset['category'] = pd.cut( titanic_dataset['age']  , 3 , labels = ['청소년' , '성인' , '노인'])
display( titanic_dataset )


cut , qcut 이용해서 나이에대한 범주를 4개로 만들어서 새로운 피처 추가한다면? - 


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,청소년
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,성인
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,청소년
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,성인
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,성인
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,성인
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,청소년
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,NaN
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,청소년


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,청소년
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,노인
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,성인
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,노인
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,노인
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,성인
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,청소년
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,NaN
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,성인


In [110]:
print()
print('[문제01] - ')
print('성별, 선실, 나이그룹(age_category) 의한 생존율을 데이터프레임으로 생성하고')
print('행에는 성별 및 나이그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다 -')
print('생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다. -')
print('cut - pivot_table')

titanic_dataset['age_category'] = pd.cut( titanic_dataset['age']  , 3 , labels = ['청소년' , '성인' , '노인'])
display( titanic_dataset )

frm = titanic_dataset.pivot_table(index   = ['sex' , 'age_category'] ,
                                  columns = 'pclass' , 
                                  values  = 'survived' , 
                                  aggfunc = 'count' , 
                                  margins = True ,
                                  margins_name = 'survived_ratio')
display( frm )

survived_frm = frm / frm.loc['survived_ratio' , 'survived_ratio'][0] 
display( survived_frm )



[문제01] - 
성별, 선실, 나이그룹(age_category) 의한 생존율을 데이터프레임으로 생성하고
행에는 성별 및 나이그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다 -
생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다. -
cut - pivot_table


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category,age_category
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,청소년,청소년
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,노인,성인
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,성인,청소년
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,노인,성인
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,노인,성인
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,성인,성인
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,청소년,청소년
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,NaN,NaN
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,성인,청소년


pclass                         1    2    3  survived_ratio
sex            age_category                               
female         청소년            29   30   68             127
               성인             47   41   33             121
               노인              9    3    1              13
male           청소년            15   37  140             192
               성인             64   53  107             224
               노인             22    9    6              37
survived_ratio               186  173  355             714

pclass                              1         2         3  survived_ratio
sex            age_category                                              
female         청소년           0.032548  0.033670  0.076319        0.142536
               성인            0.052750  0.046016  0.037037        0.135802
               노인            0.010101  0.003367  0.001122        0.014590
male           청소년           0.016835  0.041526  0.157127        0.215488
               성인            0.071829  0.059484  0.120090        0.251403
               노인            0.024691  0.010101  0.006734        0.041526
survived_ratio               0.208754  0.194164  0.398429        0.801347

In [116]:
print()
print('문제[02] - ')
print('성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다. - ')


frm = titanic_dataset.pivot_table(index   = 'sex',
                                  columns = 'pclass' , 
                                  values  = 'survived' , 
                                  aggfunc = 'count' , 
                                  margins = True ,
                                  margins_name = 'survived_ratio')

display( frm )


survived_frm =  frm / titanic_dataset.shape[0]
display( survived_frm )


문제[02] - 
성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다. - 


pclass,1,2,3,survived_ratio
sex,,,,
female,94,76,144,314
male,122,108,347,577
survived_ratio,216,184,491,891


pclass,1,2,3,survived_ratio
sex,,,,
female,0.105499,0.085297,0.161616,0.352413
male,0.136925,0.121212,0.389450,0.647587
survived_ratio,0.242424,0.206510,0.551066,1.000000


In [115]:
titanic_dataset.shape[0]

891